In [1]:
import logging
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from datasets import load_dataset
import os

2023-07-11 09:51:24.734534: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-11 09:51:25.195633: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/root/miniconda3/envs/tflow/lib/:/root/miniconda3/envs/tflow/lib/python3.10/site-packages/nvidia/cudnn/lib
2023-07-11 09:51:25.197828: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/root/miniconda3/envs

In [2]:
DATA_DIR = "./data" 
TRAIN_SIZE = 500_000

if not os.path.exists(DATA_DIR):
  !mkdir -p $DATA_DIR
  !wget "https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2016/mono/es.txt.gz" -O $DATA_DIR/OpenSubtitles.txt.gz
  !gzip -d $DATA_DIR/OpenSubtitles.txt.gz
  !head -n $TRAIN_SIZE $DATA_DIR/OpenSubtitles.txt > $DATA_DIR/train_data.txt 
  !rm $DATA_DIR/OpenSubtitles.txt

/bin/bash: /root/miniconda3/envs/tflow/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /root/miniconda3/envs/tflow/lib/libtinfo.so.6: no version information available (required by /bin/bash)
--2023-07-11 09:51:26--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2016/mono/es.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1859673728 (1.7G) [application/gzip]
Saving to: ‘./data/OpenSubtitles.txt.gz’

./data/OpenSubtitle 100%[===================>]   1.73G  32.5MB/s    in 66s     

2023-07-11 09:52:33 (26.8 MB/s) - ‘./data/OpenSubtitles.txt.gz’ saved [1859673728/1859673728]

/bin/bash: /root/miniconda3/envs/tflow/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /root/miniconda3/envs/tflow/lib/libtinfo.so.6: no version information a

In [3]:
# #load a chunk of the dataset from hugging face
# dataset = load_dataset('./spanish_billion_words.py')['train'].select(range(75_000))
# # save the dataset to a text file
# with open("dataset.txt", "w", encoding="utf-8") as file:
#     for example in dataset:
#         file.write(example["text"] + "\n")

vocab_size = 6000
tokenizer = Tokenizer(BPE())
# Configure the trainer
trainer = BpeTrainer(special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "\n"], vocab_size=vocab_size)
files = ['/mnt/c/Users/UrkoAllende/OneDrive - AMC/learning-projects/Tensorflow/Transformers/dataset.txt']
files = ['/mnt/c/Users/UrkoAllende/OneDrive - AMC/learning-projects/Tensorflow/Transformers/data/train_data.txt']


with open(files[0], 'r') as f:
    text = f.read()

tokenizer.train(files, trainer)
tokenizer.save("my_custom_bpe_tokenizer.json")

tokens = tokenizer.encode(text).ids
len(tokens)

4584392

In [4]:
tokens[:10]

[304, 3562, 70, 34, 5, 304, 1595, 577, 34, 5]

In [5]:
# Set sequence length and split ratio for validation set
sequence_length = 48
val_split = 0.1
batch_size = 512

# Create xs and ys
xs = []
ys = []
for i in range(len(tokens) - sequence_length):
    xs.append(tokens[i:i + sequence_length])
    ys.append(tokens[i+1 : i+1+ sequence_length])

# Convert xs and ys to numpy arrays
xs = np.array(xs)
ys = np.array(ys)

val_size = int(len(xs) * val_split)

# Split the data into training and validation sets
x_train, x_val = xs[:-val_size].astype(np.int32), xs[-val_size:].astype(np.int32)
y_train, y_val = ys[:-val_size].astype(np.int32), ys[-val_size:].astype(np.int32)

del xs, ys

print(f'Training size: {x_train.shape, y_train.shape}')
print(f'Validation size: {x_val.shape, y_val.shape}')

def cast_to_int32(features, labels):
    features = (tf.cast(features[0], tf.int32), tf.cast(features[1], tf.int32))
    labels = tf.cast(labels, tf.int32)
    return features, labels

train_dataset = tf.data.Dataset.from_tensor_slices(((x_train, x_train), y_train)).shuffle(buffer_size=10000).batch(batch_size)
train_dataset = train_dataset.map(cast_to_int32)

val_dataset = tf.data.Dataset.from_tensor_slices(((x_val, x_val), y_val)).batch(batch_size)
val_dataset = val_dataset.map(cast_to_int32)

del x_train, x_val, y_train, y_val

# Take a single sample from the dataset
sample = train_dataset.take(1)

for data in sample:
    features = data[0][0]
    labels = data[1]
    print("Features:", features)
    print("Labels:", labels)

Training size: ((4125910, 48), (4125910, 48))
Validation size: ((458434, 48), (458434, 48))


2023-07-11 09:54:05.176156: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 792174720 exceeds 10% of free system memory.
2023-07-11 09:54:05.313200: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 792174720 exceeds 10% of free system memory.
2023-07-11 09:54:05.411578: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 792174720 exceeds 10% of free system memory.
2023-07-11 09:54:05.555955: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 792174720 exceeds 10% of free system memory.
2023-07-11 09:54:05.632814: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 792174720 exceeds 10% of free system memory.


Features: tf.Tensor(
[[ 249 4775 1978 ...   34    5   38]
 [3997  130   19 ... 1079 4776    5]
 [ 130   34    5 ...   34    5  176]
 ...
 [  19    5   54 ...    5 4585  154]
 [  19    5   48 ...   34    5  875]
 [3260  605 1401 ...   19    5 2211]], shape=(512, 48), dtype=int32)
Labels: tf.Tensor(
[[4775 1978  252 ...    5   38 2849]
 [ 130   19    5 ... 4776    5  823]
 [  34    5 4236 ...    5  176  247]
 ...
 [   5   54 3331 ... 4585  154  754]
 [   5   48  130 ...    5  875 1917]
 [ 605 1401  170 ...    5 2211 2606]], shape=(512, 48), dtype=int32)


In [6]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 

  return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x
  
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x
  
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.25):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x) 
    return x
  
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x
class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.25):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                             d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None

  def call(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)
    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context)

    # The shape of x is (batch_size, target_seq_len, d_model).
    return x
  
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.25):
    super().__init__()

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    context, x  = inputs
    x = self.decoder(x, context)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      # b/250038731
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
  

def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0
  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)


In [7]:
num_layers = 4
d_model = 128
dff = 128 * 4
num_heads = 4
dropout_rate = 0.4
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=vocab_size,
    target_vocab_size=vocab_size,
    dropout_rate=dropout_rate)

transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

output = transformer((features, labels))
transformer.summary()

Model: "transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder (Decoder)           multiple                  2352128   
                                                                 
 dense_8 (Dense)             multiple                  774000    
                                                                 
Total params: 3,126,128
Trainable params: 3,126,128
Non-trainable params: 0
_________________________________________________________________


In [8]:
transformer.fit(train_dataset,
                epochs=1,
                validation_data=val_dataset)

8059/8059 [==============================] - 2420s 300ms/step - loss: 3.6351 - masked_accuracy: 0.3957 - val_loss: 5.8066 - val_masked_accuracy: 0.2556


In [9]:
def generate_text(transformer, tokenizer, prompt, max_length=300):
    input_tokens = tokenizer.encode(prompt).ids

    # while len(input_tokens) < sequence_length:
    #     input_tokens.append(0)
    
    input_tensor = tf.convert_to_tensor([input_tokens])
    
    for i in range(max_length):
        logits = transformer([input_tensor[:, -sequence_length:], input_tensor[:, -sequence_length:]])[0, -1]
        
        probs = tf.nn.softmax(logits)
        sampled_token = np.random.choice(len(probs), p=probs.numpy())
        
        input_tokens.append(sampled_token)
        input_tensor = tf.convert_to_tensor([input_tokens])

    pieces = [tokenizer.id_to_token(token_id) for token_id in input_tokens]
    text = ''.join([piece.replace('▁', ' ') for piece in pieces])
    
    return text

prompt = "Asegúrate de pronunciar estas frases con el acento español correcto."  
generated_text = generate_text(transformer, tokenizer, prompt, max_length=500)
print(generated_text)

Asegúrate de pronunciar estas frases con el acento español correcto.
Quisiera ser discreda.
Eso es bueno.
No puede salir el autobus que dices.
Un buen chiste.
Creo que yo diría que no
Buena eleccionaré y decides salir a Gunther.
"Este chico es algo herido y una cita, así que dijiste que eras tan tarde a la madre.
Excepto las manos en el mundo.
No tiré a punto de reír.
Y tu tambien.
- Esta bien.
Por favor, Gracias.
Espara, pero no puedo tragar cosas.
- ¿Quien?
Quedense hallar cuentas, asi que propondi una chada para allá.
- Creo que fue mandona.
- Si, creo que necesitas...
- Yo ya no opuesto.
- Ya ve.
Están aquí.
- De acuerdo.
Hay buen dinero.
- Buenas noches, Babbette.
- Hola, chicas.
- Hola, chicos.
- Hola, chicas.
- No, gracias.
- ¿Quién te dijo algo?
- Max.
Lo se.
Por favoria Sookie, lo ha hecho.
- Jamas, el Sr. Medina.
- ¿Que?
¿Mi hermano es dolor de Majestada?
Es gené.
Si, lo sé. ¿alguien elegiste?
Leía la temporada por una hora de la luego.
También eres mandona...
¿Y quién es man